# Notebook 1 — Chatbot FAQ / QA (sin retrieval)

**Enfoque:** *Prompt engineering* + *few-shot* usando ejemplos de FAQs del curso.

Este cuaderno **no** usa embeddings ni bases vectoriales. El bot responde apoyándose en un conjunto de **preguntas–respuesta** (FAQ) que cargaremos desde un archivo **JSON** o **CSV**.

## Objetivos
- Construir un Chatbot (QA): Crear un bot funcional que responda preguntas basándose en un conjunto de datos cargado (un JSON o CSV de Preguntas y Respuestas).
- Demostrar "Few-Shot Learning": Enseñar cómo "guiar" a un LLM dándole ejemplos de cómo responder (en lugar de RAG).
- Usar Búsqueda Léxica: Introducir difflib como un método para encontrar las preguntas más "parecidas" basándose en cómo están escritas (léxico).

> Requisitos: API key de OpenAI (`OPENAI_API_KEY`).

## 0) Instalación y configuración

In [1]:
# ⬇️ Dependencias mínimas
!pip install -qU langchain langchain-openai langchain-core gradio pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requ

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ''  # ← Reemplaza por tu clave
print('API key configurada (longitud):', len(os.environ.get('OPENAI_API_KEY','')))

API key configurada (longitud): 164


## 1) Cargar dataset de FAQs (JSON o CSV)
Sube un archivo con columnas/keys `question` y `answer`.

**Formato JSON esperado:**
```json
[ {"question": "¿Cuál es la fecha límite?", "answer": "El 15 de mayo."}, ... ]
```
**Formato CSV esperado:** columnas `question,answer`.


In [3]:
from google.colab import files
import io, json, pandas as pd

uploaded = files.upload()  # Sube un JSON o CSV con question/answer
fn = list(uploaded.keys())[0] if uploaded else None
print('Archivo subido:', fn)

faq_df = None
if fn:
    if fn.lower().endswith('.json'):
        data = json.loads(uploaded[fn].decode('utf-8'))
        faq_df = pd.DataFrame(data)
    elif fn.lower().endswith('.csv'):
        faq_df = pd.read_csv(io.BytesIO(uploaded[fn]))
    else:
        print('Formato no reconocido; se usará un dataset de ejemplo.')

if faq_df is None:
    # Dataset de ejemplo si no se subió nada o hubo formato inválido
    faq_df = pd.DataFrame([
        {"question": "¿Cuántas prácticas de laboratorio hay?", "answer": "Seis prácticas obligatorias a lo largo del semestre."},
        {"question": "¿Dónde se entregan los informes?", "answer": "En el campus virtual, sección Laboratorios."},
        {"question": "¿Cuál es la ponderación del examen final?", "answer": "El examen final pondera el 40% de la nota final."},
        {"question": "¿Qué software se usa en la materia?", "answer": "Principalmente Python con bibliotecas de ciencia de datos."},
        {"question": "¿Hay repositorio de materiales?", "answer": "Sí, en el repositorio institucional del curso (link en el aula virtual)."}
    ])

faq_df = faq_df[['question','answer']].dropna().reset_index(drop=True)
faq_df.head()

Saving PreguntasFAQ (1).json to PreguntasFAQ (1).json
Archivo subido: PreguntasFAQ (1).json


,question,answer
0,¿Cuál es la duración total del curso?,30 horas.
1,¿Qué nivel tiene el curso?,Nivel Avanzado.
2,¿Cuándo es la primera sesión del taller?,"El miércoles 22 de octubre, de 4:00 p.m. a 7:0..."
3,¿Quién imparte la primera sesión sobre 'Arquit...,Luis Daniel Benavides.
4,¿Qué se debe generar en la primera sesión?,Una Guía PDF con pasos de instalación y config...


## 2) Selección automática de ejemplos *few-shot*
Para no exceder el contexto, elegimos **3 preguntas** de la FAQ más **parecidas** a la consulta del usuario usando `difflib` (coincidencia aproximada **lexical**, sin embeddings). Esto **no** es retrieval semántico: solo nos ayuda a elegir **ejemplos de demostración** para el prompt.

In [10]:
import difflib
from typing import List, Dict

def pick_few_shots(user_q: str, df, k: int = 3) -> List[Dict[str,str]]:
    questions = df['question'].tolist()
    # obtener las más similares lexicalmente
    # usamos ratio aproximado con SequenceMatcher
    scored = []
    print ("\n--- 1. Calculando scores en preguntas----")
    for i, q in enumerate(questions):
        score = difflib.SequenceMatcher(None, user_q.lower(), q.lower()).ratio()
        print(f"Score: {score:.4f} | Pregunta: {q} | Índice: {i}")
        scored.append((score, i))

    scored.sort(reverse=True)
    print(f"\n---2. Preguntas ordenas por score más alto---")
    for score, i in scored:
      # Usamos questions[i] para obtener el texto de la pregunta
      print(f"Score: {score:.4f} | Pregunta: {questions[i]}")

    idxs = [i for _, i in scored[:k]]
    print(f"\n---3. Mostramos 3 con escore más alto--- ")
    for score, i in scored[:k]: #Iteramos solo sobre los 'k' primeros
      print(f"\nScore{score:.4f} | Pregunta: {questions[i]}")

    return [{"question": df.iloc[i]['question'], "answer": df.iloc[i]['answer']} for i in idxs]

## 3) Construir el *PromptTemplate* y el LLM
El prompt incluye **instrucciones claras** y **ejemplos few-shot**. Si la respuesta **no** está en la FAQ, instruimos al modelo a decirlo explícitamente.

In [45]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

SYSTEM_INSTRUCTIONS = (
    "Eres un asistente académico del curso. Responde de forma amable. "
    "Usa exclusivamente la información de las preguntas frecuentes (FAQ) proporcionadas en los ejemplos. "
    "Si la respuesta no está en la FAQ, responde: 'No se encuentra en la FAQ disponible. Pon atención al uso de sinónimos'"
)

FEW_SHOT_TEMPLATE = (
    """
{system}

A continuación verás ejemplos de preguntas frecuentes (FAQ) y sus respuestas. Úsalos como guía. Considera que las preguntas pueden tener errores ortográficos o gramaticales

### Ejemplos FAQ
{few_shot_block}
### Fin de ejemplos

Usuario: {user_question}
Asistente:
"""
)

prompt = PromptTemplate(
    template=FEW_SHOT_TEMPLATE,
    input_variables=["system", "few_shot_block", "user_question"],
)

def format_few_shot_block(examples):
    lines = []
    for ex in examples:
        lines.append(f"- Pregunta: {ex['question']}\n  Respuesta: {ex['answer']}")
    return "\n".join(lines)

def make_llm():
    try:
        return ChatOpenAI(model="gpt-4o-mini", temperature=0)
    except Exception:
        return ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = make_llm()

def answer_faq(user_q: str, df):
    # Exact match rápido
    m = df[df['question'].str.strip().str.lower() == user_q.strip().lower()]
    if len(m) > 0:
        print("\nLa pregunta es exacta, su respuesta es---")
        return m.iloc[0]['answer']

    # Few-shot por similitud lexical
    print("\n--- Proceso: Buscando Few-Shots Lexicales ---")
    fews = pick_few_shots(user_q, df, k=3)
    block = format_few_shot_block(fews)

    # Renderiza el prompt final
    rendered = prompt.format(system=SYSTEM_INSTRUCTIONS,
                             few_shot_block=block,
                             user_question=user_q
                             )

    #Se llama al LLM
    resp = llm.invoke(rendered)


    print("\n---4. Respuesta del LLM---")
    print(f"\nPara la pregunta del usuario: {user_q}")
    print("\nLa mejor respuesta es:")
    return resp.content.strip()

## 4) Prueba rápida en una celda
Escribe una pregunta y observa la respuesta del bot. Recuerda: si no está en la FAQ, debe indicarlo explícitamente.

In [38]:
print('\n---')
print("\nInicio proceso de respuesta---")
print(answer_faq("pro 12 nov", faq_df))


---

Inicio proceso de respuesta---

--- Proceso: Buscando Few-Shots Lexicales ---

--- 1. Calculando scores en preguntas----
Score: 0.2128 | Pregunta: ¿Cuál es la duración total del curso? | Índice: 0
Score: 0.1667 | Pregunta: ¿Qué nivel tiene el curso? | Índice: 1
Score: 0.1600 | Pregunta: ¿Cuándo es la primera sesión del taller? | Índice: 2
Score: 0.1389 | Pregunta: ¿Quién imparte la primera sesión sobre 'Arquitecturas y APIs'? | Índice: 3
Score: 0.1538 | Pregunta: ¿Qué se debe generar en la primera sesión? | Índice: 4
Score: 0.2000 | Pregunta: ¿Quién es el profesor del Módulo 3 'Automatización docente'? | Índice: 5
Score: 0.0351 | Pregunta: ¿En qué fecha se ve la actividad 'Bot con RAG'? | Índice: 6
Score: 0.0984 | Pregunta: ¿La sesión de 'Bot con RAG' es una práctica guiada? | Índice: 7
Score: 0.1389 | Pregunta: ¿Qué material se debe generar para la actividad 'Bot con RAG'? | Índice: 8
Score: 0.1587 | Pregunta: ¿Qué plataforma se sugiere para el 'Chatbot con FAQ'? | Índice: 9
Sco

## 4.1 Ejercicio

***Análisis de funcionamieto del ChatBot FAQ***

A) Ejecute el chunk anterior con pequeñas variaciones erradas en la pregunta, reflexione y brinde sus respuestas a las interrogantes planteadas.

**Pregunta original:** ¿Qué profesor imparte la sesión del 12 de noviembre?

**Variaciones de la pregunta:**

Qué profesor imparte la sesión del 12 de noviembre

Que profesor sesión 12 novembre

profeso 12 noviembr

profe 12 noviem

prof 12 novi


pr 12 no

Responda las siguientes preguntas

**¿Qué ha sucedido al enviar la pregunta original (sin modificaciones)?**

> Respuesta





 **¿Qué ha sucedido al ejecutar la pregunta con palabras incompletas? ¿Por qué cree que sucede?**

> Respuesta


**¿Cuál ha sido la variación del score entre las preguntas?, ¿Por queé sucede esa variación?, ¿Qué implicaciones tiene esa variación?**


> Respuesta






In [47]:
#Ahora descomente estas líneas y ejecutelas
print('\n---')
print("\nInicio proceso de respuesta---")
print(answer_faq("¿Qué universidad genera el curso?", faq_df))


---

Inicio proceso de respuesta---

--- Proceso: Buscando Few-Shots Lexicales ---

--- 1. Calculando scores en preguntas----
Score: 0.4571 | Pregunta: ¿Cuál es la duración total del curso? | Índice: 0
Score: 0.7797 | Pregunta: ¿Qué nivel tiene el curso? | Índice: 1
Score: 0.3562 | Pregunta: ¿Cuándo es la primera sesión del taller? | Índice: 2
Score: 0.4211 | Pregunta: ¿Quién imparte la primera sesión sobre 'Arquitecturas y APIs'? | Índice: 3
Score: 0.5600 | Pregunta: ¿Qué se debe generar en la primera sesión? | Índice: 4
Score: 0.3226 | Pregunta: ¿Quién es el profesor del Módulo 3 'Automatización docente'? | Índice: 5
Score: 0.4000 | Pregunta: ¿En qué fecha se ve la actividad 'Bot con RAG'? | Índice: 6
Score: 0.2143 | Pregunta: ¿La sesión de 'Bot con RAG' es una práctica guiada? | Índice: 7
Score: 0.4632 | Pregunta: ¿Qué material se debe generar para la actividad 'Bot con RAG'? | Índice: 8
Score: 0.4419 | Pregunta: ¿Qué plataforma se sugiere para el 'Chatbot con FAQ'? | Índice: 9
Sco

## 4.2 Ejercicio

***Análisis 2 de funcionamieto del ChatBot FAQ***

A) Ejecute el chunk anterior con pequeñas variaciones erradas en la pregunta, reflexione y brinde sus respuestas a las interrogantes planteadas.

**Pregunta original:** "¿Quiénes supervisan el proyecto integrador?"

**Variaciones de la pregunta:**

supervisan proyecto integrador?

supervi proyect integra?

super proye integra?

supervisan proyecto integrador

supervi proyect integrad


Responda las siguientes preguntas

**¿Qué ha sucedido al enviar la pregunta original (sin modificaciones)?**

> Respuesta


 **¿Qué ha sucedido al ejecutar la pregunta con palabras incompletas? ¿Por qué cree que sucede?**

> Respuesta

**¿Qué ha sucedido al enviar "supervisan proyecto integrador?" y "supervisan proyecto integrador"? ¿Por qué cree que ha sucedido?**

> Respuesta


**¿Cuál ha sido la variación del score entre las preguntas?, ¿Por queé sucede esa variación?, ¿Qué implicaciones tiene esa variación?**

> Respuesta


**¿Cómo propondría "arreglar" para que el LLM funcione mejor?**


> Respuesta









## 5) Interfaz de prueba con Gradio (ChatInterface)
Interfaz sencilla tipo chat para interactuar con el bot FAQ.

👉 **Nota:** Este bot **no** realiza retrieval ni busca en documentos externos. Solo usa la **FAQ cargada** y ejemplos few-shot.

In [48]:
import gradio as gr

def faq_chat(user_message, history):
    # history es una lista de [user, assistant], no la usamos para decidir la respuesta
    # porque este bot es stateless por diseño (FAQ fija)
    return answer_faq(user_message, faq_df)

demo = gr.ChatInterface(
    faq_chat,
    title="Chatbot FAQ del curso (sin retrieval)",
    description=(
        "Asistente de preguntas frecuentes. Usa únicamente la FAQ cargada y ejemplos few-shot. "
        "Si la respuesta no está en la FAQ, el asistente lo indicará."
    ),
)
demo.launch(share=False)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>